EDAF75 - notebook for lab 1
===========================

**Author:** Christian Söderberg



## SQL queries



This lab is meant to be run as a `jupyter` notebook, you can download it from the course website (the `lab1.zip` archive contains the notebook and the database).

To start your notebook, first you have to unzip `lab1.zip`, and then start `jupyter` where it can find your unpacked files.
If you install `jupyter` using `pixi` (which is what I would recommend), then you can use the following incantation (this information is obviously unnecessary if you're already running the notebook, but can be useful if you're reading this as a regular webpage&#x2026;):

<pre><code>
$ unzip lab1.zip
$ cd lab1
$ pixi init
$ pixi add jupyterlab jupysql
$ pixi run jupyter lab
</code></pre>

This should start jupyter in a browser tab, and there you can click
`lab1.ipynb`.

If you already have installed `jupyter` in some other way, you can just start it inside the `lab1` directory.



### Background



We have a database to handle the academic achievements of students at LTH &#x2013; in it we have four tables:

<center>
<img src='./lab1-er.svg'>
</center>

-   `students`
    -   `ssn`: social security number (personnummer)
    -   `first_name`: first name (duh!)
    -   `last_name`: last name

-   `departments`
    -   `department_code`: unique code for each department, such as `eda`, `fma`, &#x2026;
    -   `department_name`: the name of the department, in Swedish

-   `courses`
    -   `course_code`: course code (like `edaf75`)
    -   `course_name`: the name of the course, in Swedish (like "Databasteknik")
    -   `department_code`: the department giving the course
    -   `level`: the course level &#x2013; can be either "G1", "G2", or "A"
    -   `credits`: the number of credits for the course (like 7.5 for `edaf75`)

-   `finished_courses`
    -   `ssn`: the `ssn` of the student who has finished a course
    -   `course_code`: the `course_code` for the finished course
    -   `grade`: the grade for the student finishing the course

Some sample data:

<pre>
<code>ssn            first&#95;name  last&#95;name</code>
<code>-----------    ----------  ---------</code>
<code>19950705-2308  Anna        Johansson</code>
<code>19930702-3582  Anna        Johansson</code>
<code>19911212-1746  Emma        Alm</code>
<code>...          ...         ...</code>
<code></code>
<code>department&#95;code  department&#95;name</code>
<code>---------------  ----------------------------------------</code>
<code>eda              Datavetenskap</code>
<code>edi              Informationsteori</code>
<code>eem              Elektrisk mätteknik</code>
<code>...              ...</code>
<code></code>
<code>course&#95;code  course&#95;name           </code> <code>department&#95;code  level  credits</code>
<code>-----------  ---------------------  ---------------  ----- </code> --&#x2013;&#x2014;
<code>EDA016       Programmeringsteknik   eda              G1     7.5</code>
<code>EDA031       C++ - programmering    eda              G2     7.5</code>
<code>EDA040       Realtidsprogrammering  eda              G2     6.0</code>
<code>...          ...                    ...              ...    ...</code>
<code></code>
<code>ssn            course&#95;code  grade</code>
<code>-----------    -----------  -----</code>
<code>19950705-2308  EITN35       5</code>
<code>19950705-2308  ESS050       3</code>
<code>19950705-2308  ETIN70       4</code>
<code>...          ...          ...</code>
</code></pre>

The tables have been created with the following SQL statements:

<pre>
<code>CREATE TABLE students (</code>
<code>  ssn          CHAR(11),</code>
<code>  first&#95;name   TEXT NOT NULL,</code>
<code>  last&#95;name    TEXT NOT NULL,</code>
<code>  PRIMARY KEY  (ssn)</code>
<code>);</code>
<code></code>
<code>CREATE TABLE departments (</code>
<code>  department&#95;code  TEXT,</code>
<code>  department&#95;name  TEXT,</code>
<code>  PRIMARY KEY      (department&#95;code)</code>
<code>);</code>
<code></code>
<code>CREATE TABLE courses (</code>
<code>  course&#95;code      CHAR(6),</code>
<code>  course&#95;name      TEXT NOT NULL,</code>
<code>  department&#95;code  TEXT,</code>
<code>  level            CHAR(2),</code>
<code>  credits          DOUBLE NOT NULL CHECK (credits > 0),</code>
<code>  PRIMARY KEY      (course&#95;code),</code>
<code>  FOREIGN KEY      (department&#95;code) REFERENCES</code> <code>departments(department&#95;code)</code>
<code>);</code>
<code></code>
<code>CREATE TABLE finished&#95;courses (</code>
<code>  ssn           CHAR(11),</code>
<code>  course&#95;code   CHAR(6),</code>
<code>  grade         INTEGER NOT NULL CHECK (grade >= 3 AND grade</code> <= 5),
<code>  PRIMARY KEY   (ssn, course&#95;code),</code>
<code>  FOREIGN KEY   (ssn) REFERENCES students(ssn),</code>
<code>  FOREIGN KEY   (course&#95;code) REFERENCES</code> <code>courses(course&#95;code)</code>
<code>);</code>
</pre>

All courses offered at the "Computer Science and Engineering" program at LTH during the academic year 2013/14 are in the table `courses`.
Also, the database has been filled with made up data.
SQL statements like the following have been used to insert the data:

<pre>
<code>INTO   students(ssn, first&#95;name, last&#95;name)</code>
<code>VALUES ('19950705-2308', 'Anna', 'Johansson'),</code>
<code>       ('19930702-3582', 'Anna', 'Johansson'),</code>
<code>       ('19911212-1746', 'Emma', 'Alm'),</code>
<code>       ('19910707-3787', 'Emma', 'Nilsson'),</code>
<code>       ...</code>
</pre>



### Assignments



As said above, this lab is designed to be run as a jupyter notebook.
If you haven't got jupyter running, you can run the sql-commands for the problems below interactively inside `sqlite3`, or as a script file with all your commands, or inside some IDE, like [DB Browser for SQLite](https://sqlitebrowser.org/).

If you're running the lab as a notebook, evaluate the following cells before you begin:



In [1]:
%load_ext sql

If we install using `pixi` as show above, we're going to use `jupysql` (instead of `ipython-sql`), and we'll get a default limit of 10 rows for any query &#x2013; to turn this limit off we can write:



In [1]:
%config SqlMagic.displaylimit = None

We can now load the lab database:



In [1]:
%sql sqlite:///lab1.sqlite

The tables `students`, `departments`, `courses` and `finished_courses` are already in your database, you can see some of their contents by running the cells below:



In [1]:
%%sql
SELECT  *
FROM    students
LIMIT   4

In [1]:
%%sql
SELECT  *
FROM    departments
LIMIT   4

In [1]:
%%sql
SELECT  *
FROM    courses
LIMIT   4

In [1]:
%%sql
SELECT  *
FROM    finished_courses
LIMIT   4

If you inadvertently change the contents of the tables, you can always
recreate the them with the following command (it must be run at the
command line):

    $ sqlite3 lab1.sqlite < create-lab1-db.sql



#### Warm up problems



The problems in this section will not be reviewed during the lab session, they're just to get you up to speed.



##### Problem 1 (practice)



What are the names (first name and last name) of all the students?



In [1]:
%%sql

Now sort the names, first by last name and then by first name:



In [1]:
%%sql

When you get it to work, experiment by listing only the 10 first students (alphabetically), then try to list only students 11-20, etc.:



In [1]:
%%sql

##### Problem 2 (practice)



What are the names of the students who were born in 1985?



In [1]:
%%sql

Hint: the `substring` function can be useful (it also goes by the name
`substr`).



##### Problem 3 (practice)



The penultimate digit in the social security number is even for females, and odd for males. List the first names of all female students in our database alphabetically.



In [1]:
%%sql

Now try to output each name only once (so, no duplicates).



In [1]:
%%sql

Now try to output the number of distinct names in the listing of female students above, using `count`. It turns out that this is a bit tricky, we need to make sure we use the word `DISTINCT` in the right place (look carefully in the [documentation](https://sqlite.org/lang_aggfunc.html)).



In [1]:
%%sql

Write a query to show the number of female students, and the number of male students. We want to get the following output:

<center>
<div>
<table rules="all">
<TR>
  <TH>Gender</TH>
  <TH>Count</TH>
</TR>
<TR>
  <TD>female</TD>
  <TD>25</TD>
</TR>
  <TR><TD>male</TD>
  <TD>47</TD>
</TR>
</table>
</div>
</center>

To do this, we can use the `CASE` - `WHEN` construct (see lecture 1) &#x2013; in this case we want to create the two categories `'female'` and `'male'`, and then find a way to count the number of rows in each category.



In [1]:
%%sql

##### Problem 4 (practice)



In the next few queries, we'll look at the results of the student with the social security number `19910101-1234` &#x2013; to make things *a lot* easier, start by creating a `VIEW` with all his results.



In [1]:
%%sql
DROP VIEW IF EXISTS ...;
CREATE VIEW ... AS

Make sure the view contains all data pertinent to the student in question (it will make the following queries very simple).

Which courses (course codes only) have been taken by the student?



In [1]:
%%sql

What are the names of these courses, and how many credits do they give?



In [1]:
%%sql

How many credits has the student taken?



In [1]:
%%sql

What is the student's grade average? It turns out that there are actually (at least) two different averages at play here:

-   the unweighted average, i.e., just the average of all the students
    grades (no matter how many credits each course gives), and

-   the [weighted average](https://en.wikipedia.org/wiki/Weighted_arithmetic_mean), where we use the credits for a course as a weight.

First the unweighted average:



In [1]:
%%sql

And then the weighted average (feel free to ask me about this during QA
sessions, if you get stuck):



In [1]:
%%sql

Hint: If you've created a proper view above, we'll get a 'table' with
one row for each course the student has finished, and each row will
contain information about grades and credits for the finished course.
If we use arithmetic operations in a select statement, and then use an
aggregate function around that operation, we'll apply the aggregate
function to each value the operation returns (so, e.g., a `sum` over a
product will be a scalar product).

Now drop the view:



In [1]:
%%sql

---



#### Review problems



##### Problem 5 - REVIEW



List the 10 first students in birthday order (not by age, just the birthday &#x2013; the birthday is a part of the social security number).

The output should begin with:

<center>
<div>
<table rules="all">
<TR>
  <TH>Ssn</TH>
  <TH>First name</TH>
  <TH>Last name</TH>
</TR>
<TR>
  <TD>19910101-1234</TD>
  <TD>Peter</TD>
  <TD>Solberg</TD>
</TR>
<TR>
  <TD>19890103-1256</TD>
  <TD>Johan</TD>
  <TD>Brattberg</TD>
</TR>
<TR>
  <TD>19950125-1153</TD>
  <TD>Magnus</TD>
  <TD>Hultgren</TD>
</TR>
<TR>
  <TD>19900129-3374</TD>
  <TD>Henrik</TD>
  <TD>Gustavsson</TD>
</TR>
<TR>
  <TD>19860206-1065</TD>
  <TD>Eva</TD>
  <TD>Hjort</TD>
</TR>
</table>
</div>
</center>

Hint: You can use the `substring` function to pick out the birthday from the `ssn`.



In [1]:
%%sql

Then list all shared birthdays, order first by how many students share the birthday, then by the birthday itself (alphabetically):



In [1]:
%%sql

The output should be:

<center>
<div>
<table rules="all">
<TR>
  <TH>Birthday</TH>
  <TH>Nbr of people</TH>
</TR>
<TR>
  <TD>0206</TD>
  <TD>3</TD>
</TR>
<TR><TD>0308</TD>
<TD>2</TD>
</TR>
<TR><TD>0819</TD>
<TD>2</TD>
</TR>
<TR><TD>0915</TD>
<TD>2</TD>
</TR>
<TR><TD>1015</TD>
<TD>2</TD>
</TR>
<TR><TD>1030</TD>
<TD>2</TD>
</TR>
</table>
</div>
</center>



##### Problem 6 &#x2013; REVIEW



How many courses are there for each level (`G1`, `G2`, and `A`)?



In [1]:
%%sql

For all levels, list all departments who have more than 5 courses at that level.



In [1]:
%%sql

The output should be something like this (but it's totally fine if you list A-courses before the G1/G2 courses).

<center>
<div>
<table rules="all">
<TR><TH>Nivå</TH>
<TH>Institution</TH>
<TH>Antal</TH>
</TR>
<TR><TD>G1</TD>
<TD>Språk och litteraturcentrum</TD>
<TD>12</TD>
</TR>
<TR><TD>G1</TD>
<TD>Elektro- och informationsteknik</TD>
<TD>6</TD>
</TR>
<TR><TD>G2</TD>
<TD>Elektro- och informationsteknik</TD>
<TD>15</TD>
</TR>
<TR><TD>G2</TD>
<TD>Datavetenskap</TD>
<TD>13</TD>
</TR>
<TR><TD>A</TD>
<TD>Elektro- och informationsteknik</TD>
<TD>26</TD>
</TR>
<TR><TD>A</TD>
<TD>Matematik</TD>
<TD>16</TD>
</TR>
<TR><TD>A</TD>
<TD>Datavetenskap</TD>
<TD>14</TD>
</TR>
<TR><TD>A</TD>
<TD>Matematisk statistik</TD>
<TD>7</TD>
</TR>
<TR><TD>A</TD>
<TD>Reglerteknik</TD>
<TD>6</TD>
</TR>
</table>
</div>
</center>

*Optional*: If you really want the A-level courses listed after the G1/G2 in the output (but you're not required to!), you can use a `WITH` statement defining a table with two columns, a level and the corresponding 'order' (so, G1 has order 1, G2 has order 2 and A has order 3), and then join in this ordering and use it for ordering the rows in the output.
To do this you can use a `WITH` with a `VALUES` body (see the SQLite documentation for the [WITH clause](https://sqlite.org/lang_with.html)).



##### Problem 7 - REVIEW



For the ten departments which offer the most total credits (for its
courses in this database) &#x2013; output the name of the department, the
total offered credits, and the number of courses, order by credits, with highest total first:



In [1]:
%%sql

The output should begin something like:

<center>
<div>
<table rules="all">
<TR><TH>Institution</TH>
<TH>Totalpoäng</TH>
<TH>Antal kurser</TH>
</TR>
<TR><TD>Elektro- och informationsteknik</TD>
<TD>332.0</TD>
<TD>47</TD>
</TR>
<TR><TD>Datavetenskap</TD>
<TD>216.0</TD>
<TD>32</TD>
</TR>
<TR><TD>Matematik</TD>
<TD>160.0</TD>
<TD>26</TD>
</TR>
<TR><TD>Språk och litteraturcentrum</TD>
<TD>106.5</TD>
<TD>13</TD>
</TR>
<TR><TD>Matematisk statistik</TD>
<TD>84.0</TD>
<TD>11</TD>
</TR>
<TR><TD>...</TD>
<TD>...</TD>
<TD>...</TD>
</TR>
</table>
</div>
</center>



##### Problem 8 - REVIEW



Which students (`ssn` and full name) have taken 0 credits &#x2013; list the students in alphabetical order (last name and then first name)?
This problem can be solved in several ways, first do it with an outer join:



In [1]:
%%sql

Do the same thing using a subquery:



In [1]:
%%sql

Now just copy/paste one of your solutions above, and use a *window function* to add a column which shows the 'age order' of all the students in the list, the oldest student should get number 1 (but still be listed in alphabetical order).



In [1]:
%%sql

The output should be something like:

<center>
<div>
<table rules="all">
<TR><TH>Personnummer</TH>
<TH>Namn</TH>
<TH>Åldersordning</TH>
</TR>
<TR><TD>19891220-1393</TD>
<TD>Erik Andersson</TD>
<TD>5</TD>
</TR>
<TR><TD>19900313-2257</TD>
<TD>Erik Andersson</TD>
<TD>7</TD>
</TR>
<TR><TD>19891007-3091</TD>
<TD>Roger Brorsson</TD>
<TD>4</TD>
</TR>
<TR>
<TD>...</TD>
<TD>...</TD>
</TR>
<TR><TD>19850517-2597</TD>
<TD>Filip Persson</TD>
<TD>1</TD>
</TR>
</table>
</div>
</center>



##### Problem 9 - REVIEW



List the names and average grades of the 10 students with the highest
grade average?
You can use the unweighted average.



In [1]:
%%sql

The output should begin with:

<center>
<div>
<table rules="all">
<TR><TH>Personnummer</TH>
<TH>Förnamn</TH>
<TH>Efternamn</TH>
<TH>Snittbetyg</TH>
</TR>
<TR><TD>19861103-2438</TD>
<TD>Bo</TD>
<TD>Ek</TD>
<TD>4.35</TD>
</TR>
<TR><TD>19910308-1826</TD>
<TD>Helena</TD>
<TD>Troberg</TD>
<TD>4.31</TD>
</TR>
<TR><TD>19931213-2824</TD>
<TD>Elaine</TD>
<TD>Robertson</TD>
<TD>4.24</TD>
</TR>
<TR><TD>19930702-3582</TD>
<TD>Anna</TD>
<TD>Johansson</TD>
<TD>4.23</TD>
</TR>
<TR><TD>19931208-3605</TD>
<TD>Ylva</TD>
<TD>Jacobsson</TD>
<TD>4.22</TD>
</TR>
<TR><TD>...</TD>
<TD>...</TD>
<TD>...</TD>
<TD>...</TD>
</TR>
</table>
</div>
</center>

It's OK to use more digits in the output, but using the `format`-function it's pretty easy to get the numbers formatted as above.



##### Problem 10 - REVIEW



List the social security number and total number of credits for all
students &#x2013; order by total credits, descending.
Students with no credits
should be included in the listing, with 0 credits (not `NULL`).

Use an outer `JOIN` to solve the problem &#x2013; you might want to use the
function `coalesce(v1, v2, ...)`; it returns the first value which is
not `NULL`, so `coalesce(avg(grade), 0)` would give 0 if the were no
grades (i.e., if `grade` were `NULL`), you can also try the `ifnull`
function.



In [1]:
%%sql

##### Problem 11 - REVIEW



Do all students have unique names (first name *and* last name)? If not,
show the full name and social security number for all students who have
a namesake.

As usual there are several ways of solving this, solve it using a
`WITH`-statement where you create a 'table' with all duplicate names,
and then:

Use a `JOIN`:



In [1]:
%%sql

Use a subquery:



In [1]:
%%sql